In [2]:
import os, sys, numpy as np, pandas as pd, tensorflow as tf, re, codecs, json, time
import pickle, collections, random, math, numbers, scipy.sparse as sp, itertools, shutil

def reload(mName):
    import importlib
    if mName in sys.modules:
        del sys.modules[mName]
    return importlib.import_module(mName)

from sklearn.metrics import roc_curve, auc, confusion_matrix
from sklearn.preprocessing import minmax_scale
from sklearn.preprocessing import LabelEncoder
from matplotlib import pyplot as plt
from collections import OrderedDict, Counter
plt.style.use('ggplot')

utils = reload('utils.utils')
from utils.utils import *

np.set_printoptions(precision=4, suppress=True, linewidth=100)
randomSeed = 88
np.random.seed(randomSeed)

/Users/YiHsien/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


<br/>
<br/>
<br/>
## Data Prepare
- mapping user id, movie id to zero start indices

In [35]:
ratings, movies, uidEnc, midEnc, nUsers, nMovies, midMap, tr, te, trRatingMat, teRatingMat = prepare()

### 以leave one out方式產生 train data, test data
1. 每一筆資料有兩部分: [user query] + [item meta]
2. movie多加了兩個欄位: avg_rating, year
    - avg_rating是所有user對於同一部電影的平均評分
    - year則是從title extract出來的資訊
3. 每一筆user query 包含所有user movie history, 除了當前的rating movie (candidate movie)
4. test data的user query來自於train data

In [16]:
print(tr.shape,te.shape,ratings.shape)

(69399, 4) (30605, 4) (100004, 4)


In [56]:
movie_trans.head()

,movieId,title,genres,avg_rating,year
0,0,Toy Story (1995),"[5, 13, 10, 1, 9]",0.749438,0.815789
1,1,Jumanji (1995),"[5, 10, 9]",0.644860,0.815789
2,2,Grumpier Old Men (1995),"[1, 3]",0.591337,0.815789
3,3,Waiting to Exhale (1995),"[1, 0, 3]",0.418803,0.815789
4,4,Father of the Bride Part II (1995),[1],0.615079,0.815789


In [4]:
movie_trans, genres_enc = utils.doMovies(movies)
movie_trans["avg_rating"] = ratings.groupby("movieId").rating.mean()
movie_trans["avg_rating"] = minmax_scale(movie_trans.avg_rating.fillna(ratings.rating.mean()))
movie_trans["year"] = movie_trans.title.str.findall("\(\s*(\d+)\s*\)").map(lambda lst: int(lst[-1]) if len(lst) else None)
movie_trans["year"] = minmax_scale(movie_trans.year.fillna(movie_trans.year.median()))
n_genres = len(genres_enc.enc)

trProcessed = loo_preprocess(tr, movie_trans)
teProcessed = loo_preprocess(te, movie_trans, tr, is_train=False)
trProcessed.head()

,user_id,query_movie_ids,genres,avg_rating,year,candidate_movie_id,rating
0,0,"[1515, 1083, 833, 859, 30, 1111, 906, 1017, 10...",[0],0.835749,0.763158,931,4.0
1,0,"[931, 1083, 833, 859, 30, 1111, 906, 1017, 104...","[4, 6, 2]",0.782609,0.605263,1515,4.0
2,0,"[931, 1515, 833, 859, 30, 1111, 906, 1017, 104...","[9, 7, 3, 2]",0.621795,0.789474,1083,3.5
3,0,"[931, 1515, 1083, 859, 30, 1111, 906, 1017, 10...","[13, 10, 0, 14]",0.711640,0.342105,833,3.0
4,0,"[931, 1515, 1083, 833, 30, 1111, 906, 1017, 10...",[2],0.676768,0.824561,859,3.0


In [57]:
trProcessed.shape

(69399, 7)

In [ ]:
teProcessed.head()

### Data Function
1. 由於tensorflow placeholder不支援變動長度的columns, 需透過padding zero(補零)帶入
2. 每個變動長度的column, 需要再給lens描述每一筆資料的長度, ex: genres, genres_len

In [ ]:
def dataFn(data, n_batch=128, shuffle=False):
    def fn():
        dataInner = data.copy()
        indices = get_minibatches_idx(len(dataInner), n_batch, shuffle=shuffle)
        for ind in indices:
            yield do_multi(dataInner.iloc[ind], ["query_movie_ids", "genres"])
    return fn

for i, e in enumerate(dataFn(trProcessed, n_batch=5, shuffle=True)(), 1):
    # print(e)
    break
pd.DataFrame(e)

## MF + DNN

In [ ]:
class ModelMfDNN(object):
    def __init__(self, n_items, n_genres, dim=32, learning_rate=0.01, reg=0.05, modelDir="./model/model_mf_with_history"):
        self.n_items = n_items
        self.n_genres = n_genres
        self.dim = dim
        self.learning_rate = learning_rate
        self.reg = reg
        self.modelDir = modelDir
        graph = tf.Graph()
        with graph.as_default():
            # inputs parts
            graph_inputs(self)
            # embedding parts
            graph_embedding(self)
            # deep neural network
            graph_dnn(self)
            # computation parts
            graph_computation(self)
            # loss parts
            graph_loss(self)
            self.saver = tf.train.Saver(tf.global_variables())
            self.graph = graph
        
    def fit(self, sess, trainGen, testGen, reset=False, nEpoch=50):
        return fit(self, sess, trainGen, testGen, reset=reset, nEpoch=nEpoch)
    
    def predict(self, sess, user_queries, items):
        self.ckpt(sess, self.modelDir)
        return sess.run(self.pred, feed_dict={
            self.isTrain: False,
            self.user_id: user_queries["user_id"],
            self.query_movie_ids: user_queries["query_movie_ids"],
            self.query_movie_ids_len: user_queries["query_movie_ids_len"],

            self.genres: items["genres"],
            self.genres_len: items["genres_len"],
            self.avg_rating: items["avg_rating"],
            self.year: items["year"],
            self.candidate_movie_id: items["candidate_movie_id"]
        })
    
    def resetModel(self, modelDir):
        """刪除model dir"""
        shutil.rmtree(path=modelDir, ignore_errors=True)
        os.makedirs(modelDir)
        
    def ckpt(self, sess, modelDir):
        """load latest saved model"""
        latestCkpt = tf.train.latest_checkpoint(modelDir)
        if latestCkpt:
            self.saver.restore(sess, latestCkpt)
        return latestCkpt
    
    def feed_dict(self, data, mode="train"):
        ret = {
            self.user_id: data["user_id"],
            self.query_movie_ids: data["query_movie_ids"],
            self.query_movie_ids_len: data["query_movie_ids_len"],
            self.genres: data["genres"],
            self.genres_len: data["genres_len"],
            self.avg_rating: data["avg_rating"],
            self.year: data["year"],
            self.candidate_movie_id: data["candidate_movie_id"]
        }
        ret[self.isTrain] = False
        if mode != "infer":
            ret[self.rating] = data["rating"]
            if mode == "train":
                ret[self.isTrain] = True
            elif mode == "eval":
                pass
        return ret

    def epochLoss(self, sess, dataGen, tpe="rmse"):
        totLoss, totCnt = 0, 0
        for data in dataGen():
            lossTensor = self.rmse_loss if tpe == "rmse" else self.mae_loss
            loss = sess.run(lossTensor, feed_dict=self.feed_dict(data, mode="eval"))
            totLoss += (loss ** 2 if tpe == "rmse" else loss) * len(data["query_movie_ids"])
            totCnt += len(data["query_movie_ids"])
        return np.sqrt(totLoss / totCnt) if tpe == "rmse" else totLoss / totCnt
    
    def evaluateRMSE(self, sess, dataGen):
        """計算root mean square error"""
        self.ckpt(sess, self.modelDir)
        return self.epochLoss(sess, dataGen, tpe="rmse")

    def evaluateMAE(self, sess, dataGen):
        """計算 mean absolutely error"""
        self.ckpt(sess, self.modelDir)
        return self.epochLoss(sess, dataGen, tpe="mae")

## 設定Inputs placeholders

In [ ]:
def graph_inputs(self):
    # inputs/id_user:0
    with tf.variable_scope("inputs"):
        self.isTrain = tf.placeholder(tf.bool, None)
        # user data
        self.user_id = tf.placeholder(tf.int32, [None])
        self.query_movie_ids = tf.placeholder(tf.int32, [None, None])
        self.query_movie_ids_len = tf.placeholder(tf.int32, [None])
        # item data
        self.genres = tf.placeholder(tf.int32, [None, None])
        self.genres_len = tf.placeholder(tf.int32, [None])
        self.avg_rating = tf.placeholder(tf.float32, [None])
        self.year = tf.placeholder(tf.float32, [None])
        self.candidate_movie_id = tf.placeholder(tf.int32, [None])
        self.rating = tf.placeholder(tf.float32, [None])

## Embedding部分 = One Hot Encoding + Linear Transformation<br/>
### tensorflow 提供 tf.nn.embedding_lookup function
1. user and item 都使用embedding
2. **user bias, item bias 個別使用query_emb, item_repr projection取得, projection方式即使用tensorflow tf.matmul運算即可**

In [ ]:
def graph_embedding(self):
    init_fn = tf.glorot_normal_initializer()
    emb_init_fn = tf.glorot_uniform_initializer()
    self.b_global = tf.Variable(emb_init_fn(shape=[]), name="b_global")
    with tf.variable_scope("embedding"):
        self.w_query_movie_ids = tf.Variable(emb_init_fn(shape=[self.n_items, dim]), name="w_query_movie_ids")
        self.b_query_movie_ids = tf.Variable(emb_init_fn(shape=[dim]), name="b_query_movie_ids")
        self.w_candidate_movie_id = tf.Variable(init_fn(shape=[self.n_items, dim]), name="w_candidate_movie_id")
        self.b_candidate_movie_id = tf.Variable(init_fn(shape=[dim + 8 + 2]), name="b_candidate_movie_id")
        self.w_genres = tf.Variable(emb_init_fn(shape=[self.n_genres, 8]), name="w_genres")

        # query_movie embedding
        '''sqrtn aggregation(pooling), X: data, W: weight
               X_1*W_1 + X_2*W_2 + ... + X_n*W_n / sqrt(W_1**2 + W_2**2 + ... W_n**2)
             = weighted sum of X and normalized W
           here data = self.query_emb, weight = query_movie_mask '''
        self.query_emb = tf.nn.embedding_lookup(self.w_query_movie_ids, self.query_movie_ids)
        query_movie_mask = tf.expand_dims(tf.nn.l2_normalize(tf.to_float(tf.sequence_mask(self.query_movie_ids_len)), 1), -1)
        self.query_emb = tf.reduce_sum(self.query_emb * query_movie_mask, 1)
        self.query_bias = tf.matmul(self.query_emb, self.b_query_movie_ids[:, tf.newaxis])

        # candidate_movie embedding
        self.candidate_emb = tf.nn.embedding_lookup(self.w_candidate_movie_id, self.candidate_movie_id)

        # genres embedding
        '''sqrtn aggregation(pooling), X: data, W: weight
               X_1*W_1 + X_2*W_2 + ... + X_n*W_n / sqrt(W_1**2 + W_2**2 + ... W_n**2)
             = weighted sum of X and normalized W
           here data = self.genres_emb, weight = genres_mask '''
        self.genres_emb = tf.nn.embedding_lookup(self.w_genres, tf.to_int32(self.genres))
        genres_mask = tf.expand_dims( tf.nn.l2_normalize(tf.to_float(tf.sequence_mask(self.genres_len)), 1), -1)
        self.genres_emb = tf.reduce_sum(self.genres_emb * genres_mask, 1)

## DNN: Item(Movie Encoding)

In [ ]:
def graph_dnn(self):
    with tf.variable_scope("dnn"):
        init_fn = tf.glorot_uniform_initializer()
        # encode [item embedding + item metadata]
        self.item_repr = tf.concat([self.candidate_emb, self.genres_emb, self.avg_rating[:, tf.newaxis], self.year[:, tf.newaxis]], 1)
        self.candidate_bias = tf.matmul(self.item_repr, self.b_candidate_movie_id[:, tf.newaxis])
        
        # Do: 目前兩層DNN, 可以試著增加或減少hidden layer, 但是最後要跟query_emb內積, 最後一層維度必須 = dim
        # Example:
            # self.item_repr = tf.layers.dense(self.item_repr, 64, kernel_initializer=init_fn, activation=tf.nn.relu)
            # self.item_repr = tf.layers.dense(self.item_repr, 32, kernel_initializer=init_fn, activation=tf.nn.relu)
            # self.item_repr = tf.layers.dense(self.item_repr, self.dim, kernel_initializer=init_fn, activation=tf.nn.relu)

        # Do: 若有overfitting後可嘗試dropout
        # dp_scale = 0.5
        self.item_repr = tf.layers.dense(self.item_repr, self.dim, kernel_initializer=init_fn, activation=tf.nn.relu)
        # self.item_repr = tf.layers.dropout(self.item_repr, dp_scale, training=self.isTrain)
        self.item_repr = tf.layers.dense(self.item_repr, self.dim, kernel_initializer=init_fn, activation=tf.nn.relu)
        # self.item_repr = tf.layers.dropout(self.item_repr, dp_scale, training=self.isTrain)

## Computation: 帶公式
$$ u_i \cdot m_j + b_u + b_m + b_{global} $$

1. self.infer與self.pred節點是帶同樣的公式, 但是self.pred節點快上許多

In [ ]:
def graph_computation(self):
    with tf.variable_scope("computation"):
        infer = tf.reduce_sum(self.query_emb * self.item_repr, 1, keep_dims=True)
        infer = tf.add(infer, self.b_global)
        infer = tf.add(infer, self.query_bias)
        self.infer = tf.add(infer, self.candidate_bias, name="infer")

        # one query for all items
        self.pred = tf.matmul(self.query_emb, tf.transpose(self.item_repr)) + \
                    tf.reshape(self.candidate_bias, (1, -1)) + self.query_bias + self.b_global

## Loss
1. #### **這裡沒有使用 regularization term, (optinal! 也可自己補上)**
2. 使用MSE優化整個model
3. RMSE and MAE loss節點並不在training graph中, 主要在eval的時候可以觀察變化

In [ ]:
def graph_loss(self):
    with tf.variable_scope("loss"):
        self.loss = tf.losses.mean_squared_error(labels=self.rating[:, tf.newaxis], predictions=self.infer)
        # for eval
        self.rmse_loss = tf.sqrt(self.loss)
        self.mae_loss = tf.reduce_mean(tf.abs(self.infer - self.rating[:, tf.newaxis]))

    with tf.variable_scope("train"):
        # Do: 嘗試不同的Optimizer
        self.train_op = tf.train.GradientDescentOptimizer(learning_rate).minimize(self.loss)
        # self.train_op = tf.train.MomentumOptimizer(learning_rate, 0.9, use_nesterov=True).minimize(self.loss)
        # self.train_op = tf.train.AdagradOptimizer(learning_rate).minimize(self.loss)
        # self.train_op = tf.train.RMSPropOptimizer(learning_rate).minimize(self.loss)
        # self.train_op = tf.train.AdamOptimizer(learning_rate).minimize(self.loss)

## Fit: Training Function
1. reset: if True, clean the checkpoints save data
2. 計算epoch loss: 每個batch的loss * batch數量總和後除以epoch總數
3. training過程中會記錄valid loss, 只會儲存最低的loss => 另一種對付overfitting的方式

In [ ]:
def fit(self, sess, trainGen, testGen, reset=False, nEpoch=50):
    sess.run(tf.global_variables_initializer())
    if reset:
        print("reset model: clean model dir: {} ...".format(self.modelDir))
        self.resetModel(self.modelDir)
    # try: 試著重上次儲存的model再次training
    self.ckpt(sess, self.modelDir)

    start = time.time()
    print("%s\t%s\t%s\t%s" % ("Epoch", "Train Error", "Val Error", "Elapsed Time"))
    minLoss = 1e7
    for ep in range(1, nEpoch + 1):
        tr_loss, tr_total = 0, 0
        for i, data in enumerate(trainGen(), 1):
            loss, _ = sess.run([self.rmse_loss, self.train_op], feed_dict=self.feed_dict(data, mode="train"))
            tr_loss += loss ** 2 * len(data["query_movie_ids"])
            tr_total += len(data["query_movie_ids"])
            print("\rtrain loss: {:.3f}".format(loss), end="")
        if testGen is not None:
            epochLoss = self.epochLoss(sess, testGen)

        tpl = "\r%02d\t%.3f\t\t%.3f\t\t%.3f secs"
        if minLoss > epochLoss:
            tpl += ", saving ..."
            self.saver.save(sess, os.path.join(self.modelDir, 'model'), global_step=ep)
            minLoss = epochLoss

        end = time.time()
        print(tpl % (ep, np.sqrt(tr_loss / tr_total), epochLoss, end - start))
        start = end
    return self

## Train

In [ ]:
# hyper parameters
n_batch = 128
# Do: 嘗試不同的learning_rate [0.1, 0.001, 0.0001]
learning_rate = 0.0001
# Do: 嘗試不同的dim [8, 16, 20, 32]
dim = 3
# 非必要: 改動model dir
modelDir = "./model/model_mf_with_dnn"
    
tf.reset_default_graph()
model = ModelMfDNN(
            n_items=nMovies,
            n_genres=n_genres,
            dim=dim,
            learning_rate=learning_rate,
            modelDir=modelDir)

In [ ]:
with tf.Session(graph=model.graph) as sess:
    model.fit(sess, dataFn(trProcessed, n_batch=n_batch, shuffle=True), dataFn(teProcessed, n_batch=n_batch), nEpoch=10, reset=True)

### Another Data Function For Predict: user_item_data
1. For fast prediction, make another function
2. 原始的dataFn是 user meta + item meta, 若預測671 users對上9125 items的評分需要671 x 9125筆資料, serving time會太久
3. 現在預測所有users對所有items的資料只需要671 + 9125筆資料, 執行graph中的pred節點即可

In [ ]:
for u_data, items in user_item_data(trProcessed, np.arange(0, 5), movie_trans, n_batch=5):
    break
pd.DataFrame(u_data)

In [ ]:
pd.DataFrame(items).head()

### 單一user rating分布圖

In [ ]:
from sklearn.preprocessing import minmax_scale

# user id from 0 ~ 670
uid = 22
u_queries, movies_meta = list(user_item_data(trProcessed, [uid], movie_trans))[0]
with tf.Session(graph=model.graph) as sess:
    pred = model.predict(sess, u_queries, movies_meta)
print("shape: ", pred.shape, minmax_scale(pred.T).T)

nnzCoord = teRatingMat[uid].nonzero()
f, ax = plt.subplots(1, 2, figsize=(10, 5))
ax[0].set_title("pred distribution")
pd.Series(pred.ravel()[nnzCoord]).hist(bins=30, ax=ax[0])
ax[1].set_title("real distribution")
pd.Series(map(lambda e: e, teRatingMat[uid][nnzCoord])).hist(bins=30, ax=ax[1])
plt.show()

<br/>
<br/>
<br/>
## evaluate model

In [ ]:
%%time
n_batch = 128
with tf.Session(graph=model.graph) as sess:
    mae_ = model.evaluateMAE(sess, dataFn(teProcessed, n_batch=n_batch, shuffle=False))
    rmse_ = model.evaluateRMSE(sess, dataFn(teProcessed, n_batch=n_batch, shuffle=False))

print()
print("MAE loss: ", mae_)
print("RMSE loss: ", rmse_)

#### User導向評估(Recall)

In [ ]:
# 可給定user id細看每個user的rating與model預測效果
# valid user id from 0 ~ 670
uid = 22
with tf.Session(graph=model.graph) as sess:
    u_queries, movies_meta = list(user_item_data(trProcessed, [uid], movie_trans))[0]
    recomm = model.predict(sess, u_queries, movies_meta).ravel()
recommDf = pd.DataFrame(data={
              "userId": uid,
              "movieId": range(len(recomm)), 
              "title": midMap[np.arange(len(recomm))].values, 
              "rating": teRatingMat[uid, range(len(recomm))],
              "predRating": recomm},
             columns=("userId", "movieId", "title", "rating", "predRating"))
# ascending 可以調整True or False觀察結果
recommDf.query("rating != 0").sort_values("rating", ascending=False).head(50)

#### Model導向評估(Precision)

In [ ]:
# .query("rating != 0")
recommDf.query("rating != 0").sort_values("predRating", ascending=False).head(50)

#### ROC_CURVE (Receiver operating characteristic), AUC (Area Under Curve)

In [ ]:
coord = teRatingMat.nonzero()
with tf.Session(graph=model.graph) as sess:
    predMat = []
    for u_data, items in user_item_data(teProcessed, np.arange(nUsers), movie_trans, n_batch=128):
        predMat.append(model.predict(sess, u_data, items))
        # predMat.append(model.predict(sess, u_data))
    predMat = np.vstack(predMat)
# regard rating >= 4 as user like this movie
drawRocCurve((teRatingMat[coord] >= 4).astype(int), predMat[coord])

In [ ]:
pos_ary, neg_ary = [], []
for label in teRatingMat:
    label = label[label != 0]
    pos_ary.append(sum(label >= 4))
    neg_ary.append(sum(label < 4))
    # print("pos: {}, neg: {}".format(sum(label >= 4), sum(label < 4)))
    
def draw_pos_neg(idx):
    pd.DataFrame(
        index=idx,
        data={"pos": np.array(pos_ary)[idx], "neg": np.array(neg_ary)[idx]}).plot.bar(figsize=(10, 5), alpha=0.8)
    plt.show()

# 可調整index觀察各user rating分布 ex: [0:10] [10:20] [600:610]
draw_pos_neg(np.arange(len(teRatingMat))[0:10])

### About Test Data Movie Ratings(觀察上圖)
1. 0號, 2號, 5號, 9號 user 正向評價數量 < 10, 就算model全部預測命中, 命中率也不會是 100%!
    ex: 0號user只有1個正向評價, 全部命中也只得到0.1的分數
2. 3號user正向評價是負向評價的5倍多, 就算亂猜, 中的機率也很高

In [ ]:
_ = sum(np.sum(teRatingMat >= 4, 1) < 10)
print("{} 個user正向評價總數小於10!".format(_))

In [ ]:
print("rating數量 >= 10 且 負評價數量 >= 正評價數量 有 [{}] 人".format(sum(strict_condition(label) for label in teRatingMat)))
print("rating正評價數量 >= 0 且 rating負評價數量 >= 0 有 [{}] 人".format(sum(norm_condition(label) for label in teRatingMat)))

#### Precision At K: 
> **預測分數高(rating >= 4)的前10部電影, 和實際user rating比較, 觀察命中率**

In [ ]:
%%time
from sklearn.metrics import precision_score

n_batch = 128
with tf.Session(graph=model.graph) as sess:
    pred_mat = []
    for u_data, i_data in user_item_data(teProcessed, np.arange(nUsers), movie_trans, n_batch=n_batch):
        pred_mat.append(model.predict(sess, u_data, i_data))
    pred_mat = np.vstack(pred_mat)
    
print( "strict condition precision at 10: ", precision_at_k(teRatingMat, pred_mat, strict_condition, k=10) )
print( "norm condition precision at 10: ", precision_at_k(teRatingMat, pred_mat, norm_condition, k=10) )

### NDCG: Normalized Discounted Cumulative Gain
1. A measure of ranking quality.
2. loop 每一位user, prediciton score排序後計算NDCG
    <br/>$$ DCG_p = \sum^p_{i = 1} \frac{2^{rel_i} - 1}{log_2(i + 1)} $$<br/>
3. IDCG: Ideal DCG, 為理想狀態下的DCG分數, 即model全部命中的DCG分數, 而NDCG: Normalized DCG, 公式如下
    <br/>$$ NDCG_p = \sum^p_{i = 1} \frac{DCG_p}{IDCG_p} $$<br/>
4. 所以NDCG是一個比值, 介於0 ~ 1之間

In [ ]:
with tf.Session(graph=model.graph) as sess:
    pred_mat = []
    for u_data, items in user_item_data(teProcessed, np.arange(nUsers), movie_trans, n_batch=128):
        pred_mat.append(model.predict(sess, u_data, items))
    pred_mat = np.vstack(pred_mat)
    
strict_ndcg = all_user_ndcg(teRatingMat, pred_mat, strict_condition, label_thres=4, k=10)
norm_ndcg = all_user_ndcg(teRatingMat, pred_mat, norm_condition, label_thres=4, k=10)
print("strict condition ndcg at 10: ", strict_ndcg)
print("norm condition ndcg at 10: ", norm_ndcg)

<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
## MF + DNN with Cross Entropy Loss Function (Sigmoid Cross Entropy)

## 在Rating只有1(正向) or 0(負向)時該如何處理?
1. 繼承ModelMfDNN, 改動loss function, 因為使用sigmoid, rating欄位需要變成0 or 1
2. 每一個epoch觀察cross entropy的loss之外, 也觀察RMSE loss的變化(計算RMSE維持原rating)
3. 許多狀況很難拿到user explicit feedback, 像是movie rating, 因為沒法強迫user去rating所有movies, 
   但是應該都能夠蒐集到implicit feedback
   ex: youtube影片觀看時間長短, 點擊率, 通常implicit feedback只能歸納為到 [0, 1] 之間, 0代表negative, 1代表positive

In [ ]:
class ModelMfDNNCrossEntropy(ModelMfDNN):
    def __init__(self, *args, **kwargs):
        super(ModelMfDNNCrossEntropy, self).__init__(*args, **kwargs)
            
    def fit(self, sess, trainGen, testGen, reset=False, nEpoch=50):
        sess.run(tf.global_variables_initializer())
        if reset:
            print("reset model: clean model dir: {} ...".format(self.modelDir))
            self.resetModel(self.modelDir)
        # try: 試著重上次儲存的model再次training
        self.ckpt(sess, self.modelDir)

        start = time.time()
        print("%s\t%s\t%s\t%s\t%s" % ("Epoch", "XEntropy Error", "Val XENT Error", "Val RMSE Error", "Elapsed Time"))
        minLoss = 1e7
        for ep in range(1, nEpoch + 1):
            tr_loss, tr_total =  0, 0
            for i, data in enumerate(trainGen(), 1):
                loss, _ = sess.run([self.loss, self.train_op], feed_dict=self.feed_dict(data, mode="train"))
                batch_len = len(data["query_movie_ids"])
                tr_loss += loss * batch_len
                tr_total += batch_len
                print("\rtrain loss: {:.3f}".format(loss), end="")
                
            if testGen is not None:
                te_rmse_loss, te_xent_loss  = self.epochLoss(sess, testGen)

            tpl = "\r%02d\t%.3f\t\t%.3f\t\t%.3f\t\t%.3f secs"
            if minLoss > te_xent_loss:
                tpl += ", saving ..."
                self.saver.save(sess, os.path.join(self.modelDir, 'model'), global_step=ep)
                minLoss = te_xent_loss

            end = time.time()
            print(tpl % (ep, tr_loss / tr_total, te_xent_loss, te_rmse_loss, end - start))
            start = end
        return self

    def epochLoss(self, sess, dataGen):
        tot_xent_loss, tot_rmse_loss, tot_cnt = 0, 0, 0
        for data in dataGen():
            xent_loss, rmse_loss = sess.run([self.loss, self.rmse_loss], feed_dict=self.feed_dict(data, mode="eval"))
            batch_len = len(data["query_movie_ids"])
            tot_rmse_loss += rmse_loss ** 2 * batch_len
            tot_xent_loss += xent_loss * batch_len
            tot_cnt += batch_len
        return np.sqrt(tot_rmse_loss / tot_cnt), tot_xent_loss / tot_cnt
    

## DNN: Item(Movie Encoding)

In [ ]:
def graph_dnn(self):
    with tf.variable_scope("dnn"):
        init_fn = tf.glorot_uniform_initializer()
        # encode [item embedding + item metadata]
        self.item_repr = tf.concat([self.candidate_emb, self.genres_emb, self.avg_rating[:, tf.newaxis], self.year[:, tf.newaxis]], 1)
        self.candidate_bias = tf.matmul(self.item_repr, self.b_candidate_movie_id[:, tf.newaxis])
        
        # Do: 目前兩層DNN, 可以試著增加或減少hidden layer, 但是最後要跟query_emb內積, 最後一層維度必須 = dim
        # Example:
            # self.item_repr = tf.layers.dense(self.item_repr, 64, kernel_initializer=init_fn, activation=tf.nn.relu)
            # self.item_repr = tf.layers.dense(self.item_repr, 32, kernel_initializer=init_fn, activation=tf.nn.relu)
            # self.item_repr = tf.layers.dense(self.item_repr, dim, kernel_initializer=init_fn, activation=tf.nn.relu)

        # Do: 若有overfitting後可嘗試dropout
        # dp_scale = 0.5
        self.item_repr = tf.layers.dense(self.item_repr, dim, kernel_initializer=init_fn, activation=tf.nn.relu)
        # self.item_repr = tf.layers.dropout(self.item_repr, dp_scale, training=self.isTrain)
        self.item_repr = tf.layers.dense(self.item_repr, dim, kernel_initializer=init_fn, activation=tf.nn.relu)
        # self.item_repr = tf.layers.dropout(self.item_repr, dp_scale, training=self.isTrain)

## Computation: 帶公式
$$ \sigma(u_i \cdot m_j + b_u + b_m + b_{global}) $$
1. pred節點需要加上tf.nn.sigmoid function, **infer節點在tf.nn.sigmoid_cross_entropy_with_logits會加上sigmoid function, 不須加入**

In [ ]:
def graph_computation(self):
    with tf.variable_scope("computation"):
        infer = tf.reduce_sum(self.query_emb * self.item_repr, 1, keep_dims=True)
        infer = tf.add(infer, self.b_global)
        infer = tf.add(infer, self.query_bias)
        self.infer = tf.add(infer, self.candidate_bias, name="infer")

        # one query for all items
        self.pred = tf.matmul(self.query_emb, tf.transpose(self.item_repr)) + \
                    tf.reshape(self.candidate_bias, (1, -1)) + self.query_bias + self.b_global
        self.pred = tf.nn.sigmoid(self.pred)

## Loss: 將原本Regression的Loss(MSE or RMSE or MAE) 改成Sigmoid + Cross Entropy Loss

In [ ]:
def graph_loss(self):
    with tf.variable_scope("loss"):
        # if rating >= 4 then 1 else 0
        self.alter_rating = tf.to_float(self.rating >= 4)
        self.loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(
                labels=self.alter_rating[:, np.newaxis], logits=self.infer))

        # for observation
        self.rmse_loss = tf.sqrt(tf.losses.mean_squared_error(labels=self.alter_rating[:, tf.newaxis], predictions=self.infer))
        self.mae_loss = tf.reduce_mean(tf.abs(self.infer - self.alter_rating[:, tf.newaxis]))
        
    with tf.variable_scope("train"):
        self.train_op = tf.train.GradientDescentOptimizer(learning_rate).minimize(self.loss)
        # self.train_op = tf.train.AdagradOptimizer(learning_rate).minimize(self.loss)
        # self.train_op = tf.train.RMSPropOptimizer(learning_rate).minimize(self.loss)
        # self.train_op = tf.train.AdamOptimizer(learning_rate).minimize(self.loss)

## Trainging

In [ ]:
# hyper parameters
n_batch = 128
# Do: 嘗試不同的learning_rate [0.1, 0.001, 0.0001]
learning_rate = 0.0001
# Do: 嘗試不同的dim [8, 16, 20, 32]
dim = 16
# 非必要: 改動model dir
modelDir = "./model/model_mf_with_dnn_xent"

tf.reset_default_graph()
model_xent = ModelMfDNNCrossEntropy(
                n_items=nMovies,
                n_genres=n_genres,
                dim=dim,
                learning_rate=learning_rate,
                modelDir=modelDir)

In [ ]:
with tf.Session(graph=model_xent.graph) as sess:
    model_xent.fit(sess, dataFn(trProcessed, n_batch=n_batch, shuffle=True), dataFn(teProcessed, n_batch=n_batch), nEpoch=10, reset=True)

#### ROC_CURVE (Receiver operating characteristic), AUC (Area Under Curve)

In [ ]:
coord = teRatingMat.nonzero()
with tf.Session(graph=model_xent.graph) as sess:
    predMat = []
    for u_data, items in user_item_data(teProcessed, np.arange(nUsers), movie_trans, n_batch=128):
        predMat.append(model_xent.predict(sess, u_data, items))
    predMat = np.vstack(predMat)
# regard rating >= 4 as user like this movie
drawRocCurve((teRatingMat[coord] >= 4).astype(int), predMat[coord])

### 觀察單一user與預測分布圖

In [ ]:
from sklearn.preprocessing import minmax_scale

# user id from 0 ~ 670
uid = 22
u_queries, movies_meta = list(user_item_data(trProcessed, [uid], movie_trans))[0]
with tf.Session(graph=model_xent.graph) as sess:
    pred = model_xent.predict(sess, u_queries, movies_meta)
print("shape: ", pred.shape, minmax_scale(pred.T).T)

nnzCoord = teRatingMat[uid].nonzero()
f, ax = plt.subplots(1, 2, figsize=(10, 5))
ax[0].set_title("pred distribution")
pd.Series(pred.ravel()[nnzCoord]).hist(bins=30, ax=ax[0])
ax[1].set_title("real distribution")
pd.Series(map(lambda e: e, teRatingMat[uid][nnzCoord])).hist(bins=30, ax=ax[1])
plt.show()

#### User導向評估(Recall)

In [ ]:
# 可給定user id細看每個user的rating與model預測效果
# valid user id from 0 ~ 670
uid = 22
with tf.Session(graph=model_xent.graph) as sess:
    u_queries, movies_meta = list(user_item_data(trProcessed, [uid], movie_trans))[0]
    recomm = model_xent.predict(sess, u_queries, movies_meta).ravel()
recommDf = pd.DataFrame(data={
              "userId": uid,
              "movieId": range(len(recomm)), 
              "title": midMap[np.arange(len(recomm))].values, 
              "rating": teRatingMat[uid, range(len(recomm))],
              "predRating": recomm},
             columns=("userId", "movieId", "title", "rating", "predRating"))
# ascending 可以調整True or False觀察結果
recommDf.query("rating != 0").sort_values("rating", ascending=False).head(50)

#### Model導向評估(Precision)

In [ ]:
recommDf.query("rating != 0").sort_values("predRating", ascending=False).head(50)

### Precision At 10

In [ ]:
%%time
from sklearn.metrics import precision_score

n_batch = 128
with tf.Session(graph=model_xent.graph) as sess:
    pred_mat = []
    for u_data, i_data in user_item_data(teProcessed, np.arange(nUsers), movie_trans, n_batch=n_batch):
        pred_mat.append(model_xent.predict(sess, u_data, i_data))
    pred_mat = np.vstack(pred_mat)
    
print( "strict condition precision at 10: ", precision_at_k(teRatingMat, pred_mat, strict_condition, k=10) )
print( "norm condition precision at 10: ", precision_at_k(teRatingMat, pred_mat, norm_condition, k=10) )

### NDCG

In [ ]:
with tf.Session(graph=model_xent.graph) as sess:
    pred_mat = []
    for u_data, items in user_item_data(teProcessed, np.arange(nUsers), movie_trans, n_batch=128):
        pred_mat.append(model_xent.predict(sess, u_data, items))
    pred_mat = np.vstack(pred_mat)
    
strict_ndcg = all_user_ndcg(teRatingMat, pred_mat, strict_condition, label_thres=4, k=10)
norm_ndcg = all_user_ndcg(teRatingMat, pred_mat, norm_condition, label_thres=4, k=10)
print("strict condition ndcg at 10: ", strict_ndcg)
print("norm condition ndcg at 10: ", norm_ndcg)

<br/>
<br/>
<br/>
## 取出movies embedding, 使用cosine similarity列出最相似的電影

In [ ]:
movies[movies.title.str.contains("Toy")]

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances

def most_like(model, seed_movie, k=10):
    """給定某一部電影, 使用model裡movies embedding找尋cosine相似度高的其他電影!"""
    with tf.Session(graph=model.graph) as sess:
        model.ckpt(sess, model.modelDir)
        user_queries, items = list(user_item_data(trProcessed, [0], movie_trans))[0]
        movie_emb = sess.run(model.item_repr, feed_dict={
            model.isTrain: False,
            model.genres: items["genres"],
            model.genres_len: items["genres_len"],
            model.avg_rating: items["avg_rating"],
            model.year: items["year"],
            model.candidate_movie_id: items["candidate_movie_id"]
        })
        
    most_like = cosine_similarity(movie_emb[seed_movie][np.newaxis, :], movie_emb).ravel().argsort()[::-1][:k]
    return movies.iloc[most_like]

# mse訓練出來的model
most_like(model, 0, k=11)

In [ ]:
# cross entropy訓練出來的model
most_like(model_xent, 0, k=11)

<br/>
<br/>
<br/>

## Observations
1. 可觀察到使用movie embedding以cosine相似度找相似的movie已經有效果, 但是我們用的並不是自己組合出來的movie features, 而是embedding, 代表movie帶入features所train出來的embedding是有效果的
